In [ ]:
from rdkit import Chem
import pandas as pd
import numpy as np
from sympy import Polygon


from importlib import reload
import py3Dmol
import rcm
file_path_connectivity_table = 'cp6t6_0p_blyp35_geom_connectivity_global.csv'
xyz_file_path = 'cp6t6_0p_blyp35_geom.xyz'

probably store xyz in pandas as x, y, z insted of numpy array. one can than easily add extra column of the B fileds associated

In [ ]:
a = rcm.RCM(xyz_file_path,file_path_connectivity_table)
mol = a.mol
mol = Chem.RWMol(mol)
a.get_B(0,0,0)

In [ ]:
from rcm import area

In [ ]:
coords_3d = np.array([
            [ 0,  0, 0],
            [ 2,  0, 0],
            [ 2,  2, 0],
            [ 0,  2, 0],
            [ 0, -2, 0],
            [-2, -2, 0],
            [-5,  0, 0],
            ])

In [ ]:
area.area_3d(coords_3d)

In [ ]:
no_split_path = a.conn.current_weight == 1

In [ ]:
sum(no_split_path)

In [ ]:
df_temp = a.conn.loc[no_split_path,::]

In [ ]:
df_temp.iloc[0,::].start

In [ ]:
df_temp.iloc[0,::].end

In [85]:
temp_index_depo = []

temp_index = int(df_temp.iloc[0,::].end)
temp_index_depo.append(temp_index)

while len(temp_index_depo) < 20:
    temp_index = int(df_temp.loc[df_temp.start == temp_index,'end'])
    temp_index_depo.append(temp_index)


TypeError: cannot convert the series to <class 'int'>

In [86]:
temp_index

53

In [87]:
temp_index_depo

[139, 147, 65, 59, 53]

In [88]:
df_temp.loc[df_temp.start == temp_index,'end']

Series([], Name: end, dtype: int64)

In [ ]:
# xyz_view = py3Dmol.view(
#     data=Chem.MolToMolBlock(mol), 
#     style={"stick": {}, "sphere": {"scale": 0.25}},
#     width=400,height=400,
#     )
# xyz_view.zoomTo()

In [ ]:
# #Calculate area
# import pyny3d.geoms as pyny

# coords_3d = np.array([
#                     [0,  0, 0],
#                     [2,  0, 0],
#                     [2,  0, 2],
#                     [0,  0, 2]
#                     ])
# polygon = pyny.Polygon(coords_3d)
# print(f'Area is : {polygon.get_area()}')